# Collection: copublications internationales (UE et hors UE)
* demande interne INRIA (27/07/2025)
* Réalisation du script (adaptation d'un ancien script) : Kumar Guha (Data DCIS/Inria)
* Date 27/02/2025, modifié le 29/08/2025. Denière version : 15/09/2025.

## Choix
* On ne retient que la première affiliation de chaque auteur (pas les niveaux supérieurs : exemple : Boston University School of Medicine et pas Boston University).
* Si un auteur rattaché à une structure française est aussi rattaché à une structure étrangère, on ne retient pas cet auteur pour compter une copublication internationale.
* Si un auteur de la structure Inria recherchée est aussi affilié à une autre strucutre étrangère, celle-ci n'est pas mentionnée.


## Étapes
* Extraire les publications des équipes concernées.
* identifier les publications dont les auteurs sont affiliés à un organisme étranger (hors France et DOM TOM)
* On crée des listes d'identifiants uniques pour les affiliations FR, Union Européenne et hors UE.
    *  on exclut les organismes étrangers dont les auteurs sont aussi affiliés à une structure FR
    *  on exclut les affiliations en double pour une même publication
    *  nettoyage des données.
* Génération d'un fichier Excel avec : chiffres, liste des publications, liste des organismes étrangers copubliants
* Première identification de la ville d'après les données de nom de structure et d'adresse.


## Extraction des publications de HAL


In [ ]:
# 
###############################################################
## Extraction des publications de HAL
##############################################
import requests
import time
import os
import pandas as pd
from datetime import datetime, timedelta
import logging
from lxml import etree
import gc 
import locale
import re

###############################################################################################
##################### variables à modifier avant lancement du script ##########################
###############################################################################################

###########################################################
# Définir la structure recherchée:
###########################################################
nom_collection = "INRIA" # il s'agit des publications des équipes Inria de tous les centres.

# Identifiant de la structure "de réference" dont on analyse les publications (exemple Centre Inria de Rennes : 419153)
#(à chercher dans Aurehal : https://aurehal.archives-ouvertes.fr/structure/index)
id_aurehal_de_la_structure = "34586" #34586 = Centre Inria Université Côte d'Azur
nom_de_la_structure = "Sophia"

# Structures à exclure
# Galinette, Stack (id Aurehal :1088569,495900, 1088566, 525233 )
equipes_a_exclure = []

##########################################
# Définir la période recherchée
###########################################
annee_debut = 2018
annee_fin = 2024 # indiquer la même année si la recherche porte sur une seule année



#####################################################################
##################### script ########################################
#####################################################################
# Obtenir la date actuelle
date_extraction_current = datetime.now().strftime("%Y-%m-%d")

## Spécifier le répertoire de log
log_directory = '../log/'
## Créer le répertoire s'il n'existe pas
os.makedirs(log_directory, exist_ok=True)


# Configuration du logger
log_file = date_extraction_current + '__international_publications_log.txt'
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

# Configurer la localisation en français
locale.setlocale(locale.LC_TIME, "French_France.1252")

# La période est définie par les années saisies dans les variables au-dessus du script

# periode = "[" + annee_debut + " TO " + annee_fin + "]"

# variables cumulatives

all_dataex = {}
all_datafr = {}
all_datapubli = []
params = {}
pas = 3

for start_year in range(annee_debut, annee_fin + 1, pas):
    end_year = min(start_year + pas - 1, annee_fin)
    periode = f"[{start_year} TO {end_year}]"
    
    print(f"▶️ Traitement de la période : {periode}")
    
    params["q"] = f"publicationDateY_i:{periode}"
    
    # Réinitialise tes variables internes ici si besoin :
    cursor_mark = "*"
    previous_cursor_mark = None
    compteur = 0
    partenaire = 0
    dataex = []
    datafr = []
    datapubli = []
    unique_org_ex = {}
    unique_org_fr = {}


    # Fonction permettant de réessayer s'il n'y a pas de réponse
    def fetch_with_retry(url, params=None, max_retries=3, delay=2):
        """
        Effectue une requête GET avec plusieurs tentatives en cas d'échec.
        
        Args:
            url (str): L'URL de la requête.
            params (dict, optional): Paramètres de requête.
            max_retries (int): Nombre maximal de tentatives.
            delay (int): Temps d'attente entre chaque tentative (en secondes).

        Returns:
            Response object si la requête réussit, sinon None.
        """
        for attempt in range(max_retries):
            try:
                response = requests.get(url, params=params, timeout=10)  # Timeout pour éviter les blocages
                
                if response.status_code == 200:
                    return response  # Succès
                
                print(f"⚠️ Tentative {attempt + 1} échouée ({response.status_code}). Nouvelle tentative...")

            except requests.RequestException as e:
                print(f"⏳ Erreur réseau ({e}), tentative {attempt + 1}...")


    #### Processus de récupération de la liste des notices présentes dans HAL pour la période spécifiée
    # URL de base de l'API
    base_url = f"https://api.archives-ouvertes.fr/search/{nom_collection}?"

    # Paramètres de la requête : les résultats sont traités un par un en xml-tei
    params = {
        "q": f"publicationDateY_i:{periode}",
        "fq": f"structId_i:{id_aurehal_de_la_structure}",
        "wt": "xml-tei",
        "rows": 1,
        "sort": "docid asc"
    }

    # https://api.archives-ouvertes.fr/search/INRIA2?q=publicationDateY_i:[2019%20TO%202024]&fq=structId_i:(413916%20OR%20526070%20OR%20526181%20OR%20521735%20OR%20521714)&wt=xml-tei&rows=100&sort=docid%20asc

    # Initialisation du cursorMark (qui permet de réitérer la requête jusqu'à la fin des réponses de l'API)
    cursor_mark = "*"
    previous_cursor_mark = None

    # Définition des variables
    compteur = 0
    compte_publisUps = 0
    partenaire = 0
    dataex = []
    datafr = []
    datapubli = []
    unique_org_ex = {}
    unique_org_fr = {}
    # Exclure les DOM-TOM français
    France_et_dom_tom_codes = ['FR','GP', 'RE', 'MQ', 'GF', 'YT', 'PM', 'WF', 'TF', 'NC', 'PF']


    namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}


    #######################################
    # La requête est lancée en boucle et obtient un résultat (une notice) à chaque fois
    # chaque résultat est traité dans la boucle "while"
    ########################################
    while cursor_mark != previous_cursor_mark:
        # Mise à jour du cursorMark
        params["cursorMark"] = cursor_mark
        #print(f"CursorMark: {cursor_mark}")
        compteur += 1
        if compteur % 5000 == 0 or compteur == 1:
            print(compteur)
            if compteur % 5000 == 0:
                # Convertir les données collectées pour les organismes étrangers
                dataex = list(unique_org_ex.values())
                df_ex = pd.DataFrame(dataex)
                # Convertir les données collectées pour les organismes français
                datafr = list(unique_org_fr.values())
                df_fr = pd.DataFrame(datafr)

                # Liste des publications/logiciels de HAL
                df_publis = pd.DataFrame(datapubli)
                # df_ex.to_excel(f"df_ex_{compteur}.xlsx", index=False)
                # df_fr.to_excel(f"df_fr_{compteur}.xlsx", index=False)
                # df_publis.to_excel(f"df_publis_{compteur}.xlsx", index=False)
            time.sleep(2)  # Pause de 2 secondes entre les requêtes
    
        # Limite pour tests
        # if compteur > 15:
        #     break

        response = fetch_with_retry(base_url, params)
        if response:
            try:
                tree = etree.fromstring(response.content)
            except etree.XMLSyntaxError:
                print("Erreur de syntaxe XML. Réponse non analysée.")
                continue

            # Récupération de la valeur de next dans l'attribut de la première balise TEI
            next_cursor_mark = tree.attrib.get("next")
            # print(next_cursor_mark)
            
            # indication du nombre de notices répondant à la requête
            quantity_value = tree.find('.//tei:measure', namespaces=namespaces).attrib.get('quantity')
            
            if cursor_mark == "*":
                # seulement lors de la première boucle, on indique le nombre total de notices répondant à la requête
                print(f"nbre résultats : {quantity_value}. Durée estimée pour 4000 notices : 20 mn")

            # identification de la notice dans le xml-tei pour trouver les métadonnées
            biblfull_elements = tree.findall('.//tei:biblFull', namespaces=namespaces)

            if biblfull_elements:
                biblfull = biblfull_elements[0]  # Get the first matching element
            else:
                biblfull = None  # Handle the absence of the element
                if cursor_mark == next_cursor_mark:
                    print(f"ancien curseur : {cursor_mark} - nouveau curseur : {next_cursor_mark}")
                    print("pas de biblFull - Terminé")
                    break
                else:
                    print(f"Il y a eu un problème dans la réponse de HAL, veuillez relancer le script")
                    break

            # Récupération des metadonnées
            if biblfull is not None:
                #identifiant de la publication dans HAL
                halID = biblfull.xpath('.//tei:publicationStmt/tei:idno[@type="halId"]/text()', namespaces=namespaces) or ["pas de hal_ID"]
                halID_value = halID[0] if halID else "no HalID"
                # print(halID_value)
                
                # Domaines = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:classCode[@scheme="halDomain"]/text()', namespaces=namespaces)
                # Domaines_value = ";".join(domaine.strip() for domaine in Domaines if domaine)


                # TRAITEMENT DES AFFILIATIONS contenues dans la notice
                orgs = tree.findall('.//tei:listOrg[@type="structures"]/tei:org', namespaces=namespaces)

                for org in orgs:
                    xml_id = org.xpath('@xml:id', namespaces=namespaces) # code de la structure
                    lenom = org.xpath('.//tei:orgName/text()', namespaces=namespaces)
                    lacronyme = org.xpath('.//tei:orgName[@type="acronym"]/text()', namespaces=namespaces)
                    lepays = org.xpath('.//tei:country/@key', namespaces=namespaces)
                    ladresse = [addr.text for addr in org.xpath('.//tei:addrLine', namespaces=namespaces) if addr.text]
                    ladresse_value = " ".join(ladresse)
                    lesrelations = org.xpath('.//tei:listRelation/tei:relation/@active', namespaces=namespaces) # codes des structures parentes

                    # Supprimer '#struct-' de chaque élément de la liste
                    lesrelations_cleaned = [relation.replace('#struct-', '') for relation in lesrelations]
                    xml_id_cleaned = xml_id[0].lstrip('struct-') 
                    
                    # si l'identifiant structure fait partie des identifiants à exclure on passe au suivant sans traiter.
                    # if xml_id_cleaned in equipes_a_exclure:
                    #     continue

                    # Organismes copubliants non français
                    if lepays and lepays[0] not in France_et_dom_tom_codes:

                        # print (f"{xml_id_cleaned} trouvé")
                        partenaire = 1 # à noter qu'il peut s'agir d'une structure mère étrangère qui ne va pas entrer en compte au final
                        unique_org_ex[xml_id[0]] = {
                            "Pays_ex": lepays,  # Le pays (on filtrera ensuite)
                            "OrganismeEx": lenom[0],  # Les noms des institutions
                            "ID_aurehal": xml_id_cleaned,  # L'attribut xml:id
                            "adresse": ladresse_value,
                            "parents": lesrelations_cleaned

                        }
                    # Organisme FR
                    elif lepays and lepays[0] in France_et_dom_tom_codes:

                        #print(lepays)
                        unique_org_fr[xml_id[0]] = {
                            "Pays_fr": lepays,  # Le pays
                            "Organisme_fr": lenom[0],  # Les noms des institutions
                            "Acronyme_fr": lacronyme[0] if lacronyme else 'na',
                            "ID_aurehal": xml_id_cleaned,  # L'attribut xml:id
                            "adresse": ladresse_value,
                            "parents": lesrelations_cleaned

                        }
                
                # Si on veut limiter les résultats aux publications avec des copubliants internationaux alors il faut décommenter la ligne suivante      
                if partenaire == 1: # si on a trouvé un pays hors FR
            
                # et il faut décaler les lignes suivantes aussi
                # Sinon, on prend toutes les publications (par ex, si on veut calculer la proportion de copublications avec l'étranger par rapport au total)

                    # Récupération de l'année de publication   
                    
                    date_value = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct//tei:monogr/tei:imprint/tei:date[@type="datePub"]/text()', namespaces=namespaces)
                    date_produced = biblfull.xpath('.//tei:editionStmt/tei:edition/tei:date[@type="whenProduced"]/text()', namespaces=namespaces)
                    if date_value and date_value is not None:
                        date_text = date_value[0]  # Récupérer la chaîne de date
                        year_value = date_text[:4]  # Les 4 premiers caractères pour l'année
                    else:
                        year_value = date_produced[0][:4]

                    keywords = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:keywords/tei:term', namespaces=namespaces)
                    # Extraire les mots-clés et joindre avec ";"
                    keywords_str = ";".join(
                        " ".join(term.text.split())  # supprime espaces multiples et trims
                        for term in keywords
                        if term.text
                    )
                    # Récupérer tous les <classCode> avec scheme="halDomain"
                    hal_domain_elems = biblfull.xpath(
                        './/tei:profileDesc/tei:textClass/tei:classCode[@scheme="halDomain"]',
                        namespaces=namespaces
                    )

                    # Nettoyer et joindre avec ";"
                    if hal_domain_elems:
                        hal_domain_str = ";".join(
                            elem.text.strip() for elem in hal_domain_elems if elem.text
                        )
                    else:
                        hal_domain_str = ""
                        
                    # Récupérer <abstract> en priorité "en", sinon "fr", sinon chaîne vide
                    def get_full_text(elem):
                        return "".join(elem.itertext()).strip() if elem is not None else ""
                    abstract_elem = biblfull.xpath(
                        './/tei:profileDesc/tei:abstract[@xml:lang="en"]',
                        namespaces=namespaces
                    )
                    if not abstract_elem:  # fallback en "fr"
                        abstract_elem = biblfull.xpath(
                            './/tei:profileDesc/tei:abstract[@xml:lang="fr"]',
                            namespaces=namespaces
                        )
                    abstract_str = get_full_text(abstract_elem[0]) if abstract_elem else ""
                    
                    # titre_revue = titre_journal[0].text if titre_journal  else ""
                    # # print(titre_revue)
                    # conference_titles = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct/tei:monogr/tei:meeting/tei:title', namespaces=namespaces)
                    # titre_conf = conference_titles[0].text if conference_titles else ""
                    # print(titre_conf)
            
                
                # Identification des affiliations associées à chaque auteur
                    for author in biblfull.xpath('.//tei:titleStmt/tei:author', namespaces=namespaces):
                        forename = author.xpath('.//tei:persName/tei:forename/text()', namespaces=namespaces)  or ["Unknown"]
                        surname = author.xpath('.//tei:persName/tei:surname/text()', namespaces=namespaces)  or ["Unknown"]
                                        
                        authorLastFirstnames = (f"{surname[0]}, {forename[0]}")

                        affiliations = author.xpath('.//tei:affiliation/@ref', namespaces=namespaces)

                        for affiliation in affiliations:
                            affiliation = affiliation.lstrip('#struct-')

                            # si l'identifiant structure fait partie des identifiants à exclure on passe au suivant sans traiter.
                            # if affiliation in equipes_a_exclure:
                            #     continue

                            datapubli.append ({
                                "halID" : halID_value,
                                "Auteur" : authorLastFirstnames,
                                "affiliation" : affiliation,
                                "Annee" : year_value,
                                "MotsCles": keywords_str,
                                "Domaine(s)":hal_domain_str,
                                "Resume":abstract_str,
                            })
                    partenaire = 0

                #print(f"{halID_value} - {authorLastFirstnames} - {affiliation}")
                    
                # En cas de récupération intensive de données, forcer la libération de la mémoire
                # gc.collect()

                # Mise à jour du cursorMark pour la prochaine itération
                previous_cursor_mark = cursor_mark
                cursor_mark = next_cursor_mark
                # Pause pour éviter de surcharger l'API
                # time.sleep(0.1)

                if not next_cursor_mark:
                    print(compteur)
                    break

    # Ajoute les résultats cumulés
    all_dataex.update(unique_org_ex)
    all_datafr.update(unique_org_fr)
    all_datapubli.extend(datapubli)

print("Terminé")

In [ ]:
######################################################################
# Conversion en "dataframes" pour traitement des données et comptage
######################################################################

df_ex = pd.DataFrame(list(all_dataex.values()))
df_fr = pd.DataFrame(list(all_datafr.values()))
df_publis = pd.DataFrame(all_datapubli)

# Sauvegarde pour contrôle et tests
# df_ex.to_excel("df_ex_total.xlsx", index=False)
# df_fr.to_excel("df_fr_total.xlsx", index=False)
# df_publis.to_excel("df_publis_total.xlsx", index=False)
print("dataframes créés")


In [ ]:
#  pour tests : df_publis=pd.read_excel("df_publis_total.xlsx")

In [ ]:
df_publis.head(3)

In [ ]:

####################################
# FILTRE UE / hors UE pour df_ex
####################################
df_nonUE = ""
df_UE = ""
pays_UE = [
    "AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR",
    "DE", "GR", "HU", "IE", "IT", "LV", "LT", "LU", "MT", "NL",
    "PL", "PT", "RO", "SK", "SI", "ES", "SE"
]

# On va créer 2 dataframes en fonction du pays de la structure (UE ou hors UE)
#

# Fonction pour vérifier si une valeur de la liste est dans la colonne `identifiant` ou `parents`
def filter_rows_EU(row):
    # Vérifie si une des valeurs de la liste se trouve dans `country` ou dans les valeurs de `parents`
    return any(country in pays_UE for country in row["Pays_ex"])

def filter_rows_ex(row):
    # Vérifie si une des valeurs de la liste ne se trouve pas dans `country` ou dans les valeurs de `parents`
    return any(country not in pays_UE for country in row["Pays_ex"])


# Filtrer les lignes du DataFrame pour garder celles des structures qui nous intéressent
df_UE = df_ex[df_ex.apply(filter_rows_EU, axis=1)]
df_UE.rename(columns={"OrganismeEx" : "Organisme_UE"}, inplace=True)


df_nonUE = df_ex[df_ex.apply(filter_rows_ex, axis=1)]
df_nonUE.rename(columns={"OrganismeEx" : "Organisme_Hors_UE"}, inplace=True)


print("Dataframes UE et non UE créés")


In [ ]:
###########################################################
# Interprétation des codes Pays en noms en toutes lettres 
###########################################################

# Récupérer les données de l'API
# url = "https://restcountries.com/v3.1/all"
# response = requests.get(url)
# countries_data = response.json()
import requests

def get_country_mapping():
    url = "https://restcountries.com/v3.1/all"
    params = {"fields": "cca2,name"}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            countries_data = response.json()
            if isinstance(countries_data, list):
                return {
                    country.get("cca2"): country.get("name", {}).get("common")
                    for country in countries_data
                    if country.get("cca2") and "name" in country and "common" in country["name"]
                }
            else:
                print("⚠️ Format inattendu :", type(countries_data))
                return {}
        else:
            print(f"⚠️ Erreur API ({response.status_code}): {response.json().get('message')}")
            return {}
    except Exception as e:
        print("❌ Erreur lors de la récupération des données pays :", e)
        return {}

# Utilisation
country_mapping = get_country_mapping()
print("✅ Exemple : FR →", country_mapping.get("FR"))  # Affiche 'France'




df_UE["Pays_ex"] = df_UE["Pays_ex"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
df_UE['TypePays'] = "EU"

df_nonUE["Pays_ex"] = df_nonUE["Pays_ex"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
df_nonUE['TypePays'] = "EX"

df_fr["Pays_fr"] = df_fr["Pays_fr"].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
df_fr['TypePays'] = "FR"


# Afficher un aperçu du DataFrame modifié
print(df_UE.head(1))



In [ ]:
########## 
# Ajouter l'acronyme des auteurs Inria dans la liste des publications
###########

# Charger ton fichier d'équipes Inria
df_equipes = pd.read_excel("equipesInriadeAurehal.xlsx")

# Vérifie que les colonnes utiles existent
assert "docid" in df_equipes.columns, "Colonne 'docid' manquante dans le fichier Excel"
assert "acronyme" in df_equipes.columns, "Colonne 'acronyme' manquante dans le fichier Excel"

# S'assurer que 'docid' et 'affiliation' sont bien de type str
df_equipes["docid"] = df_equipes["docid"].astype(str)
df_publis["affiliation"] = df_publis["affiliation"].astype(str)

# Fusion des deux DataFrames : on ajoute l'acronyme en fonction de l'affiliation
df_publis = df_publis.merge(df_equipes[["docid", "acronyme"]], how="left", left_on="affiliation", right_on="docid")

# Optionnel : supprimer la colonne docid (redondante après le merge)
df_publis.drop(columns=["docid"], inplace=True)

# Exemple d'affichage
print(df_publis[["halID", "Auteur", "affiliation", "acronyme"]].head(10))


In [ ]:
#  pour tests df_publis[df_publis["halID"] == "hal-00799242"]

In [ ]:
# Nettoyage : Pour les auteurs Inria (auteurs ayant un acronyme), supprimer les autres affiliations
# Marquer les cas où pour chaque (halID, Auteur), au moins un acronyme est non-null
df=df_publis

df["has_acronyme"] = df.groupby(["halID", "Auteur"])["acronyme"].transform(lambda x: x.notna().any())

# Ne garder que :
# - les lignes où acronyme est non-null
# - ou les cas où aucune ligne pour ce (halID, Auteur) n'a d'acronyme
df = df[(df["acronyme"].notna()) | (~df["has_acronyme"])]

# Supprimer la colonne temporaire
df = df.drop(columns=["has_acronyme"])

df_publis = df
df_publis.head(10)


In [ ]:
# Ajouter l'organisme et le pays UE des auteurs à la liste générale des publications

# Vérifier que les colonnes existent
assert "ID_aurehal" in df_UE.columns, "Colonne 'ID_aurehal' manquante dans df_UE"
assert "Organisme_UE" in df_UE.columns, "Colonne 'Organisme_UE' manquante dans df_UE"
assert "Pays_ex" in df_UE.columns, "Colonne 'Pays_ex' manquante dans df_UE"
assert "adresse" in df_UE.columns, "Colonne 'adresse' manquante dans df_UE"

# Harmoniser les types de colonnes pour le merge
df_UE["ID_aurehal"] = df_UE["ID_aurehal"].astype(str)
df_publis["affiliation"] = df_publis["affiliation"].astype(str)

# Renommer les colonnes de df_nonUE pour éviter les collisions
df_UE_renamed = df_UE.rename(columns={
    "adresse": "adresse_UE",  # Pour éviter d’écraser la précédente
})

# Faire la jointure
df_publis = df_publis.merge(
    df_UE_renamed[["ID_aurehal", "Organisme_UE", "Pays_ex","adresse_UE"]],
    how="left",
    left_on="affiliation",
    right_on="ID_aurehal"
)

# Supprimer la colonne intermédiaire redondante
df_publis.drop(columns=["ID_aurehal"], inplace=True)

# Vérification du résultat
print(df_publis[["affiliation", "Organisme_UE", "Pays_ex","adresse_UE"]].head())


In [ ]:
df_nonUE.head(1) #pour connaître le nom des colonnes afin de faire le traitement suivant


In [ ]:
# Ajouter l'organisme et le pays Hors UE des auteurs

# Vérifier que les colonnes existent
assert "ID_aurehal" in df_nonUE.columns, "Colonne 'ID_aurehal' manquante dans df_nonUE"
assert "Organisme_Hors_UE" in df_nonUE.columns, "Colonne 'Organisme_Hors_UE' manquante dans df_nonUE"
assert "Pays_ex" in df_nonUE.columns, "Colonne 'Pays_ex' manquante dans df_nonUE"
assert "adresse" in df_nonUE.columns, "Colonne 'adresse' manquante dans df_nonUE"

# Harmoniser les types de colonnes pour le merge
df_nonUE["ID_aurehal"] = df_nonUE["ID_aurehal"].astype(str)
df_publis["affiliation"] = df_publis["affiliation"].astype(str)

# Renommer les colonnes de df_nonUE pour éviter les collisions
df_nonUE_renamed = df_nonUE.rename(columns={
    "Pays_ex": "Pays_ex_horsUE",  # Pour éviter d’écraser la précédente
    "adresse": "adresse_hors_UE", 
})

# Faire la jointure
df_publis = df_publis.merge(
    df_nonUE_renamed[["ID_aurehal", "Organisme_Hors_UE", "Pays_ex_horsUE","adresse_hors_UE"]],
    how="left",
    left_on="affiliation",
    right_on="ID_aurehal"
)

# Supprimer la colonne intermédiaire redondante
df_publis.drop(columns=["ID_aurehal"], inplace=True)

# Vérification du résultat
print(df_publis[["affiliation", "Organisme_Hors_UE", "Pays_ex_horsUE","adresse_hors_UE"]].head())

In [ ]:
#  contrôle pour tests df_publis[df_publis["halID"] == "hal-00799242"]

In [ ]:
##########################################
# Supprimer les auteurs qui ont à la fois une affiliation française et une affiliation étrangère)
#########################################
# Indicateurs
df_publis["has_org"] = df_publis["Organisme_UE"].notna() | df_publis["Organisme_Hors_UE"].notna()
df_publis["has_no_acronyme"] = df_publis["acronyme"].isna() | (df_publis["acronyme"].str.strip() == "")

# Grouper par halID + Auteur
grouped = df_publis.groupby(["halID", "Auteur"]).agg(
    n_lignes=("halID", "count"),          # nombre de lignes pour ce couple
    has_no_acronyme=("has_no_acronyme", "any"),
    has_org=("has_org", "any")
).reset_index()

# On garde uniquement ceux qui ont au moins 2 lignes et les deux cas
auteurs_mixtes = grouped[
    (grouped["n_lignes"] >= 2) &
    (grouped["has_no_acronyme"]) &
    (grouped["has_org"])
]

# Supprimer ces auteurs de df_publis
df_publis_clean = df_publis.merge(
    auteurs_mixtes[["halID", "Auteur"]],
    on=["halID", "Auteur"],
    how="left",
    indicator=True
)
df_publis_clean = df_publis_clean[df_publis_clean["_merge"] == "left_only"].drop(columns="_merge")

print(f"✅ {len(df_publis) - len(df_publis_clean)} lignes supprimées")



In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "hal-00799242"]

In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "hal-01895279"]

In [ ]:
df_publis = df_publis_clean

In [ ]:
# Nettoyage : On supprime tous les co-auteurs français = ne sont pas Inria (pas d'acronyme) et n'ont pas d'affiliations étrangères

# Colonnes à tester pour le vide
cols_to_check = ["Organisme_UE", "Organisme_Hors_UE", "acronyme"]

# Masque des lignes vides
mask_empty = df_publis[cols_to_check].apply(
    lambda row: all(pd.isna(v) or str(v).strip() == "" for v in row),
    axis=1
)

# On garde seulement les lignes qui ne sont pas "vides"
df_publis_clean = df_publis[~mask_empty].copy()

print(f"✅ {mask_empty.sum()} lignes supprimées")



In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "hal-00799242"]

In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "cea-04228169"]

In [ ]:
df_publis = df_publis_clean

In [ ]:
df_publis.head(5)

In [ ]:
# On ne garde pas les publications avec juste un seul auteur
df_publis_clean = df_publis[df_publis.groupby("halID")["halID"].transform("count") > 1].copy()


In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "hal-00799242"]

In [ ]:
# contrôle pour test df_publis_clean[df_publis_clean["halID"] == "cea-04228169"]

In [ ]:
df_publis = df_publis_clean

In [ ]:
# Fichier final (1e partie) avec , pour chaque auteur Inria, les copubliants étrangers

import pandas as pd

# 1. On part du df_publis et on isole les auteurs FR et étrangers
auteurs_fr = df_publis[pd.notna(df_publis['acronyme']) & (df_publis['acronyme'].str.strip() != '')]
auteurs_etr = df_publis[pd.isna(df_publis['acronyme']) | (df_publis['acronyme'].str.strip() == '')]

# 2. Pour chaque auteur étranger, on veut rattacher les auteurs FR du même halID
rows = []
for _, row_etr in auteurs_etr.iterrows():
    hal_id = row_etr['halID']
    # Trouver les auteurs FR liés à ce halID
    fr_list = auteurs_fr[auteurs_fr['halID'] == hal_id]
    for _, row_fr in fr_list.iterrows():
        rows.append({
            'Equipe': row_fr['acronyme'],
            'Auteurs FR': row_fr['Auteur'],
            'Auteurs copubliants': row_etr['Auteur'],
            'Organisme copubliant': row_etr['Organisme_Hors_UE'] if pd.notna(row_etr['Organisme_Hors_UE']) else row_etr['Organisme_UE'],
            'Adresse': row_etr['adresse_hors_UE'] if pd.notna(row_etr['adresse_hors_UE']) else row_etr['adresse_UE'],
            'Pays': row_etr['Pays_ex_horsUE'] if pd.notna(row_etr['Pays_ex_horsUE']) else row_etr['Pays_ex'],
            'ID Aurehal': row_etr['affiliation'],
            'Année': row_etr['Annee'],
            'UE/Non UE': 'UE' if pd.notna(row_etr['Organisme_UE']) else 'Non UE',
            'HalID': hal_id,
            'Domaine(s)': row_etr['Domaine(s)'],
            'Mots-cles' : row_etr['MotsCles'],
            'Resume':row_etr['Resume'],
        })

# 3. Construire le DataFrame final
df_final = pd.DataFrame(rows)


# 4. Trier par Equipe, Auteurs FR, Auteurs copubliants
df_final = df_final.sort_values(by=['Equipe', 'Auteurs FR', 'Auteurs copubliants']).reset_index(drop=True)


# 5. Exporter vers Excel
nom_du_fichier = f"Copubliants_par_auteur_Inria_{nom_de_la_structure}.xlsx"
df_final.to_excel(nom_du_fichier, index=False)

print(f"✅ Fichier Excel créé : {nom_du_fichier}")


In [ ]:
# contrôle pour test df_final[df_final["HalID"] == "hal-00799242"]

In [ ]:
#####
# Filtre sur le centre (pour supprimer lignes d'équipes d'autres centres copubliantes)
####

import pandas as pd

print(f"je sélectionne les équipes de {nom_de_la_structure}")

# Charger ton mapping
df_equipes = pd.read_excel("EquipesInriadeAurehal.xlsx")

# On garde seulement les équipes qui ont le nom de la structure dans parent_name
equipes_centre = df_equipes[df_equipes["parent_name"].str.contains(nom_de_la_structure, case=False, na=False)]

# On récupère la liste des acronymes valides
liste_acronymes = equipes_centre["acronyme"].unique()

# print(liste_acronymes)

# Filtrer df_final pour ne garder que les Equipes présentes dans cette liste
df_final_centre = df_final[df_final["Equipe"].isin(liste_acronymes)]

# Résultat
print(df_final_centre.shape)
df_final_centre.head()

# Ajouter une colonne Centre
df_final_centre["Centre"] = nom_de_la_structure


# 5. Exporter vers Excel

df_final_centre.to_excel(f"{nom_du_fichier}", index=False)


In [ ]:

df_final_centre.head(3)

In [ ]:
# contrôle pour test print(nom_du_fichier)

In [ ]:
# pip install geotext

In [ ]:
print(nom_du_fichier)

In [ ]:
###################
# fichier final avec ville :
# repérage de la ville selon 3 méthodes successives:
#1. le mot entre crochets du titre de la structure (en excluant les pays d'après une liste de pays courants)
#2. Regex d'après le repérage d'un code postal de l'adresse 
#3. En faisant appel à l'analyse de l'adresse par le service Geotext
#####################""

import pandas as pd
from geotext import GeoText
import re

# Charger ton fichier Excel initial
df = pd.read_excel(nom_du_fichier)

# Liste des pays à ignorer
pays_a_ignorer = {
    "Algeria", "Argentina", "Australia", "Austria", "Belgium", "Bolivia", 
    "Bosnia and Herzegovina", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", 
    "Cameroon", "Canada", "Chile", "China", "Colombia", "Costa Rica", "Croatia", 
    "Cyprus", "Czechia", "Denmark", "Ecuador", "Estonia", "Finland", "Georgia", 
    "Germany", "Greece", "Hong Kong", "Hungary", "Iceland", "India", "Indonesia", 
    "Iran", "Ireland", "Israel", "Italy", "Japan", "Jordan", "Kenya", "Latvia", 
    "Lebanon", "Lithuania", "Luxembourg", "Madagascar", "Malaysia", "Malta", 
    "Mexico", "Morocco", "Netherlands", "New Zealand", "Niger", "Nigeria", 
    "North Macedonia", "Norway", "Oman", "Pakistan", "Peru", "Poland", "Portugal", 
    "Romania", "Russia", "Saudi Arabia", "Senegal", "Serbia", "Singapore", 
    "Slovakia", "Slovenia", "South Africa", "South Korea", "Spain", "Sweden", 
    "Switzerland", "Taiwan", "Thailand", "Tunisia", "Turkey", "Uganda", "Ukraine", 
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", 
    "Venezuela", "Vietnam"
}

# Fonction fusionnée
def get_ville(organisme, adresse):
    # 1. Vérifier crochets dans "Organisme copubliant"
    if isinstance(organisme, str):
        match = re.search(r"\[(.*?)\]", organisme)
        if match:
            contenu = match.group(1).strip()
            if contenu not in pays_a_ignorer:
                return contenu
    
    # 2. Sinon regex sur "Adresse"
    if isinstance(adresse, str):
        match = re.search(r"\b(\d{4,5})\s+([A-Za-zÀ-ÖØ-öø-ÿ\- ]+)", adresse)
        if match:
            city = match.group(2).strip()
            city = re.sub(r"[^A-Za-zÀ-ÖØ-öø-ÿ\- ]", "", city)
            if city:
                return city

        # 3. Sinon GeoText
        places = GeoText(adresse)
        if places.cities:
            return places.cities[0]

    return None

# Appliquer la fonction
df["Ville"] = df.apply(lambda row: get_ville(row["Organisme copubliant"], row["Adresse"]), axis=1)

# Réordonner les colonnes
cols_order = [
    'Centre','Equipe','Auteurs FR', 'Auteurs copubliants', 'Organisme copubliant',
    'Adresse', 'Ville', 'Pays', 'ID Aurehal', 'UE/Non UE', 'Année',
    'HalID','Domaine(s)','Mots-cles','Resume'
]
df = df[cols_order]

# Sauvegarder le fichier final
df.to_excel(f"Copubliants_par_auteur_Inria_{nom_de_la_structure}_ville_final.xlsx", index=False)

print("✅ Fichier créé avec détection Ville (crochets → regex → GeoText)")


In [ ]:
df.head(3)


In [ ]:
## Rendre les valeurs cliquables dans fichier excel

from openpyxl import load_workbook

nomdufichier = f"Copubliants_par_auteur_Inria_{nom_de_la_structure}_ville_final.xlsx"


# Charger le fichier Excel
wb = load_workbook(nomdufichier)
ws = wb.active

# Définir une fonction pour créer les liens
def create_link_hal(hal):
    return f'https://inria.hal.science/{hal}'

def create_link_aurehal(aurehal):
    return f'https://aurehal.archives-ouvertes.fr/structure/read/id/{aurehal}' # lien vers la structure

def create_link_revconf_conf(revconf):
    return f'https://revconf.inria.fr/conference/view/{revconf}'

def create_link_revconf_journal(revconf_journal):
    return f'https://revconf.inria.fr/journal/view/{revconf_journal}'

def create_link_revconf_editeur(revconf_editeur):
    return f'https://revconf.inria.fr/publisher/view/{revconf_editeur}'

def create_link_monitor(monitor):
    return f'https://monitor.hal.science/?structure={monitor}&publicationDate=2020-2024'

# Parcourir les cellules de la colonne 'n' (même valeur 'n' pour min_col et max_col qui est la colonne où se trouve l'identifiant ) et ajouter les liens hypertexte
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=12, max_col=12):
    for cell in row:
        hal = cell.value
        if hal:
            cell.hyperlink = create_link_hal(hal)
print ("liens hal terminé")

for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=9, max_col=9):
    for cell in row:
        aurehal = cell.value
        if aurehal:
            cell.hyperlink = create_link_aurehal(aurehal)
print ("liens aurehal terminé")


# Enregistrer les modifications dans le fichier Excel
wb.save(nomdufichier)

print("ouf, terminé 2")